In [2]:
import torch, subprocess, textwrap
print("Torch:", torch.__version__)
print("CUDA available:", torch.cuda.is_available())
print("CUDA (torch):", torch.version.cuda)
print(textwrap.shorten(subprocess.run(["nvidia-smi"], text=True, capture_output=True).stdout, width=500))






Torch: 2.3.1+cu121
CUDA available: True
CUDA (torch): 12.1
Sun Aug 10 00:13:39 2025 +-----------------------------------------------------------------------------------------+ | NVIDIA-SMI 580.65.06 Driver Version: 580.65.06 CUDA Version: 13.0 | +-----------------------------------------+------------------------+----------------------+ | GPU Name Persistence-M | Bus-Id Disp.A | Volatile Uncorr. ECC | | Fan Temp Perf Pwr:Usage/Cap | Memory-Usage | GPU-Util Compute M. | | | | MIG M. | [...]


In [5]:
!nvidia-smi


Sun Aug 10 00:14:25 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 580.65.06              Driver Version: 580.65.06      CUDA Version: 13.0     |
+-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       On  |   00000000:00:04.0 Off |                    0 |
| N/A   38C    P8             13W /   70W |       3MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [1]:
from huggingface_hub import snapshot_download
from pathlib import Path

mistral_models_path = Path.home().joinpath('mistral_models', '7B-Instruct-v0.3')
mistral_models_path.mkdir(parents=True, exist_ok=True)

snapshot_download(repo_id="mistralai/Mistral-7B-Instruct-v0.3", allow_patterns=["params.json", "consolidated.safetensors", "tokenizer.model.v3"], local_dir=mistral_models_path)


/home/namit/venv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Fetching 3 files: 100%|███████████████████████████| 3/3 [02:52<00:00, 57.64s/it]


'/home/namit/mistral_models/7B-Instruct-v0.3'

In [2]:
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
import torch

model_id = "mistralai/Mistral-7B-Instruct-v0.3"

bnb = BitsAndBytesConfig(load_in_4bit=True, bnb_4bit_compute_dtype=torch.bfloat16,
                         bnb_4bit_use_double_quant=True, bnb_4bit_quant_type="nf4")

tok = AutoTokenizer.from_pretrained(model_id, use_fast=True)
model = AutoModelForCausalLM.from_pretrained(
    model_id, device_map="auto", torch_dtype=torch.bfloat16, quantization_config=bnb
)
model.eval()


Loading checkpoint shards: 100%|██████████████████| 3/3 [01:24<00:00, 28.13s/it]


MistralForCausalLM(
  (model): MistralModel(
    (embed_tokens): Embedding(32768, 4096)
    (layers): ModuleList(
      (0-31): 32 x MistralDecoderLayer(
        (self_attn): MistralSdpaAttention(
          (q_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
          (k_proj): Linear4bit(in_features=4096, out_features=1024, bias=False)
          (v_proj): Linear4bit(in_features=4096, out_features=1024, bias=False)
          (o_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
          (rotary_emb): MistralRotaryEmbedding()
        )
        (mlp): MistralMLP(
          (gate_proj): Linear4bit(in_features=4096, out_features=14336, bias=False)
          (up_proj): Linear4bit(in_features=4096, out_features=14336, bias=False)
          (down_proj): Linear4bit(in_features=14336, out_features=4096, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): MistralRMSNorm((4096,), eps=1e-05)
        (post_attention_layernorm): MistralRMSNo

In [ ]:
import json, torch

# ---- reply-only helper (unchanged) ----
def chat_reply_only(messages, max_new_tokens=140, do_sample=False, temperature=0.0, top_p=1.0):
    text = tok.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)
    inputs = tok([text], return_tensors="pt").to(model.device)
    prompt_len = inputs["input_ids"].shape[-1]
    with torch.inference_mode():
        out = model.generate(
            **inputs,
            max_new_tokens=max_new_tokens,
            do_sample=do_sample,
            temperature=temperature,
            top_p=top_p,
            pad_token_id=tok.eos_token_id,
        )
    gen_ids = out[0][prompt_len:]
    return tok.decode(gen_ids, skip_special_tokens=True).strip()

# ---- strict instruction with 2 examples ----
SYSTEM_DETECT = """
You are an academic integrity assistant. Classify the text as HUMAN or AI GENERATED.
Return ONLY a single JSON object (no prose, no markdown, no code fences).
Schema: {"label":"HUMAN"|"AI","confidence":0-1,"rationale":"<one sentence>"}

Examples:
Text: "As an AI language model, I cannot browse the internet but I can provide..."
Output: {"label":"AI","confidence":0.85,"rationale":"Meta-disclaimer and templated phrasing"}

Text: "I collected results over three trials and wrote a limitation section about sampling bias."
Output: {"label":"HUMAN","confidence":0.78,"rationale":"Personal procedure details and idiosyncratic phrasing"}

Now classify the next text.
""".strip()

# ---- balanced JSON extractor (no regex recursion needed) ----
def extract_first_json(s: str):
    # find first '{'
    start = s.find('{')
    if start == -1:
        raise ValueError("no opening brace")
    depth = 0
    in_str = False
    esc = False
    for i in range(start, len(s)):
        ch = s[i]
        if in_str:
            if esc:
                esc = False
            elif ch == '\\':
                esc = True
            elif ch == '"':
                in_str = False
        else:
            if ch == '"':
                in_str = True
            elif ch == '{':
                depth += 1
            elif ch == '}':
                depth -= 1
                if depth == 0:
                    return json.loads(s[start:i+1])
    raise ValueError("no balanced json")


def detect_ai(text: str, attempts: int = 3):
    for i in range(attempts):
        do_sample = (i > 0)   # first try greedy, then allow mild sampling
        temp, top_p = ((0.3, 0.9) if do_sample else (0.0, 1.0))
        reply = chat_reply_only(
            [{"role":"system","content":SYSTEM_DETECT},
             {"role":"user","content":f"Text:\n{text}\n\nOutput must be valid JSON only, with double quotes and no trailing commas."}],
            max_new_tokens=150, do_sample=do_sample, temperature=temp, top_p=top_p
        )
       
        try:
            out = extract_first_json(reply)
            lab = out.get("label", "HUMAN")
            if lab not in ("HUMAN","AI"):
                lab = "HUMAN"
            conf = float(out.get("confidence", 0.5))
            conf = max(0.0, min(1.0, conf))
            rat = out.get("rationale", "")
            return {"label": lab, "confidence": conf, "rationale": rat}
        except Exception:
            continue
        return {"label": "HUMAN", "confidence": 0.5, "rationale": "Parsing failed; review manually."}

# ---- quick smoke test ----
tests = [
    "In this lab I used ggplot2 to visualize trends and discussed limitations.",
    "As an AI language model, I will now provide a comprehensive response."
]
for t in tests:
    print(t)
    print(detect_ai(t), "\n")



In this lab I used ggplot2 to visualize trends and discussed limitations.
{'label': 'HUMAN', 'confidence': 0.8, 'rationale': 'Specific software tool mention and personal lab context'} 

As an AI language model, I will now provide a comprehensive response.
{'label': 'AI', 'confidence': 0.99, 'rationale': 'Meta-disclaimer and templated phrasing'} 



 Create Dummy Dataset

In [ ]:
import pandas as pd

# Create dummy HUMAN/AI dataset
data = [
    {"text": "In this lab I used ggplot2 to visualize trends and discussed limitations.", "label": "HUMAN"},
    {"text": "As an AI language model, I will now provide a comprehensive response.", "label": "AI"},
    {"text": "Our company achieved record profits this quarter, exceeding expectations.", "label": "HUMAN"},
    {"text": "Here is a list of 10 ways to improve productivity:", "label": "AI"},
    {"text": "The experiment yielded consistent results across multiple trials.", "label": "HUMAN"},
    {"text": "I am an AI assistant programmed to help you with your queries.", "label": "AI"}
]
df = pd.DataFrame(data)



 Evaluate Model Performance

In [24]:
import time
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

preds, gts, times = [], [], []

for _, row in df.iterrows():
    start = time.time()
    out = detect_ai(row["text"])
    times.append(time.time() - start)
    preds.append(out["label"])
    gts.append(row["label"])

# Compute metrics
acc = accuracy_score(gts, preds)
prec = precision_score(gts, preds, pos_label="AI")
rec = recall_score(gts, preds, pos_label="AI")
f1 = f1_score(gts, preds, pos_label="AI")
avg_latency = sum(times) / len(times)

perf_report = {
    "n_samples": len(df),
    "accuracy": acc,
    "precision": prec,
    "recall": rec,
    "f1": f1,
    "avg_latency_sec": avg_latency
}

# Save performance report
pd.DataFrame([perf_report]).to_csv("performance_report.csv", index=False)


/home/namit/venv/lib/python3.11/site-packages/transformers/generation/configuration_utils.py:601: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.0` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(


 Cost Notes

In [ ]:
cost_notes = """
# Mistral Model Cost Notes

- **Model:** mistralai/Mistral-7B-Instruct-v0.3
- **Load in 4-bit Quantization:** Yes (nf4) — reduces VRAM cost.
- **Runtime Environment:** Google Cloud VM with NVIDIA T4 GPU.
- **Average Latency per Sample:** 4.443 sec
- **Estimated Cost:** Depends on GPU rental time; with T4 on GCP preemptible instance, ~$0.3500 per hour.

"""
